In [1]:
using System.Threading;

static Action<Thread> skip_in_console = delegate (Thread thread) { thread.Join(); };

# Потоки

Для начала, вспомним, как организовано выполнение программ на ПК. Программа представляет из себя набор инструкций, которые выполняются процессором. Процессор (CPU) состоит из устройства управления (Control Unit) и арифметико-логического устройства (Arithmetic Logic Unit). Устройство управления получает инструкции из памяти, декодирует их и передает на выполнение в арифметико-логическое устройство. После выполнения инструкции, управление возвращается в устройство управления, которое получает следующую инструкцию и передает ее на выполнение. Таким образом, процессор последовательно выполняет инструкции, которые составляют программу.

![Процессор](./images/processor.jpeg)

Как видно, процессор может выполнить одну инструкцию за раз. Однако, если мы посмотрим загрузку процессора типичного ПК, то увидим, что он не работает на 100%. При этом в системе выполняется множество программ. 

Как это возможно? Дело в том, что процессор не выполняет инструкции программы последовательно, а переключается между ними. При этом, если одна из программ заблокирована, то процессор переключается на другую программу, которая может быть выполнена. Таким образом, процессор выполняет инструкции программы не последовательно, а параллельно. 

При этом, если в системе несколько процессоров, то они могут выполнять инструкции разных программ одновременно. Такая организация выполнения программ называется многозадачностью (multitasking).

Для реализации многозадачности в операционной системе существует понятие потока (**thread**). Поток — это основная единица, которой операционная система выделяет время процессора. Упрощенно, поток представляет собой некоторую последовательность инструкций, которая выполняется "параллельно" с другими. При этом, все потоки в системе выполняются одновременно, но на самом деле процессор последовательно выполняет инструкции каждого потока. Таким образом, потоки позволяют реализовать параллельное выполнение программ.

Для разделения времени процессора между потоками, операционная система использует понятие кванта времени (**time slice**). Квант времени — это некоторый промежуток времени, в течение которого процессор выполняет инструкции одного потока. По истечении кванта времени, процессор переключается на другой поток. Таким образом, операционная система реализует многозадачность.

![Потоки](./images/threads.png)

Для разделения программ между потоками, операционная система использует понятие процесса (**process**). Процесс — это некоторая программа, которая выполняется в системе. Однако, процесс не является физическим объектом, а является абстракцией, которая представляет из себя набор ресурсов, необходимых для выполнения программы. Каждый процесс может состоять из нескольких потоков.

Различные процессы в системе выполняются независимо друг от друга. При этом, каждый процесс имеет свое адресное пространство, которое изолировано от других процессов. Таким образом, процессы не могут обращаться к памяти друг друга.

Различные технологии и фреймворки предоставляют различные способы работы с потоками. Давайте рассмотрим инструменты, которые предоставляет dotnet.

In [3]:
static void Main()
{
    Thread thread = new Thread (new ThreadStart(WriteY)); // Kick off a new thread
    thread.Start(); // running WriteY()

    // Simultaneously, do something on the main thread.
    for (int i = 0; i < 100; i++)
    {
        Console.Write ("x");
    }

    skip_in_console(thread);
}

static void WriteY()
{
    for (int i = 0; i < 100; i++)
    {
        Console.Write ("y");
    }
}

Main();

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Для создания потока в dotnet используется класс `Thread` пространства имен `System.Threading`. Единицей исполнения в потоке является делегат (метод), который передается в конструктор. При этом, выполнение инструкций в потоке начнется только после вызова метода `Start()`.

При запуске консольного приложения, в системе создается процесс, который выполняет код в методе `Main()`. При этом поток, выполняющий метод `Main()` называется основным потоком (Main Thread) . При вызове метода `Start()` инструкции, определенные в методе `WriteY()` таже начинают "параллельно" исполняться.

Процесс, запущенный при старте приложения dotnet (host process) будет изолирован от других процессов на уровне системы. Потоки же, запущенные внутри процесса, будут разделять ресурсы процесса. И в случае с dotnet, степень их изоляции будет ограничена контекстом выполнения делегата, а это в первую очередь стек вызовов и локальные переменные.

Кроме этого, в контексте потока сохраняется текущие культурные настройки, приоритет планирования потока и другие параметры. Таким образом мы можем говорить, что программа dotnet имеет столько стеков вызовов, сколько потоков в ней выполняется.

Рассмотрим на примере как это работает

In [4]:
static void Main()
{
    Thread thread = new Thread (new ParameterizedThreadStart(Write)); // Kick off a new thread
    thread.Start("y"); // running Write()

    Write("x"); // Simultaneously, do something on the main thread.    

    skip_in_console(thread);
}

static void Write(object symbol)
{
    for (int i = 0; i < 100; i++)
    {
        Console.Write($"{symbol}[{i}]");
    }
}

Main();       

x[1]x[2]x[3]x[4]x[5]x[6]x[7]x[8]x[9]x[10]x[11]x[12]x[13]x[14]x[15]x[16]x[17]x[18]x[19]x[20]x[21]x[22]x[23]x[24]x[25]x[26]x[27]x[28]x[29]x[30]x[31]x[32]x[33]x[34]x[35]x[36]x[37]x[38]x[39]x[40]x[41]x[42]x[43]x[44]x[45]x[46]x[47]x[48]x[49]x[50]x[51]x[52]x[53]x[54]x[55]x[56]x[57]x[58]x[59]x[60]x[61]x[62]x[63]x[64]x[65]x[66]x[67]x[68]x[69]x[70]x[71]x[72]x[73]x[74]x[75]x[76]x[77]x[78]x[79]x[80]x[81]x[82]x[83]x[84]x[85]x[86]x[87]x[88]x[89]x[90]x[91]x[92]x[93]x[94]x[95]x[96]y[0]y[1]y[2]y[3]y[4]y[5]y[6]y[7]x[97]x[98]x[99]y[8]y[9]y[10]y[11]y[12]y[13]y[14]y[15]y[16]y[17]y[18]y[19]y[20]y[21]y[22]y[23]y[24]y[25]y[26]y[27]y[28]y[29]y[30]y[31]y[32]y[33]y[34]y[35]y[36]y[37]y[38]y[39]y[40]y[41]y[42]y[43]y[44]y[45]y[46]y[47]y[48]y[49]y[50]y[51]y[52]y[53]y[54]y[55]y[56]y[57]y[58]y[59]y[60]y[61]y[62]y[63]y[64]y[65]y[66]y[67]y[68]y[69]y[70]y[71]y[72]y[73]y[74]y[75]y[76]y[77]y[78]y[79]y[80]y[81]y[82]y[83]y[84]y[85]y[86]y[87]y[88]y[89]y[90]y[91]y[92]y[93]y[94]y[95]y[96]y[97]y[98]y[99]

В этом примере поток создается с делегатом, который при вызове будет принимать параметр типа `object`, который является параметром запуска потока, и передается в метод `Start()`.

Поскольку значения параметров и переменных хрпняться на стеке, то все значения параметров и локальных переменных будут разными для каждого потока. В отличие от значений, хранящихся на стеке, значения, хранящиеся в куче, будут общими для всех потоков.

Такая особенность показывает, почему многопоточность полезна: один поток может получать данные в фоновом режиме, например,
в то время как другой поток может отображать данные по мере их поступления.

In [7]:
public class Program
{
    bool done;

    // Note that Go is now an instance method
    public void Go()
    {
        if (!done)
        { 
            done = true;
            Console.WriteLine("Done");
        }
    }
}

static void Main()
{
    Program tt = new Program();   // Create a common instance
    var thread = new Thread(tt.Go);
    thread.Start();
    tt.Go();

    skip_in_console(thread);
}

Main();

Done


Статические данные (значений полей) как и статические члены типов являются общими для всех потоков. Используем это свойство, однако немного поменяем  порядок вывода сообщений в консоль.

In [10]:
static bool done;    // Static fields are shared between all threads

static void Main()
{
    var thread = new Thread(Go);
    thread.Start();
    Go();

    skip_in_console(thread);
}

static void Go()
{
    if (!done)
    { 
        Console.WriteLine("Done");
        done = true; 
    }
}

Main();

Done
Done


Как видно, в этом случае, сообщение может быть выведено более одного раза. Это связано с тем, что в одном потоке, значение `done` может быть изменено после того, как другой поток проверит его значение. Такая ситуация называется **race condition** (состояние гонки) или просто конкуррентным доступом к ресурсам. 

Это классическая проблема многопоточности, которая может возникнуть, когда два или более потока имеют доступ к общему ресурсу. В этом случае, общий ресурс - это поле `done`. Для ее решения необходимо использовать механизм синхронизации доступа к ресурсу.

Один из таких механизмов применим в следующем примере:

In [11]:
static bool done;
static readonly object locker = new object();

static void Main()
{
    var thread = new Thread(Go);
    thread.Start();
    Go();

    skip_in_console(thread);
}

static void Go()
{
    lock (locker)
    {
        if (!done)
        {
            Console.WriteLine("Done");
            done = true;
        }
    }
}

Main();

Done


С помощью ключевого слова `lock` мы можем обеспечить выполнение определенного блока кода только одним потоком (по очереди). Это обеспечивается с помощью взаимоисключающей (mutually exclusive) блокировки общего ресурса. В данном случае, ресурсом является объект `locker`. В этом случае, если один поток уже захватил объект `locker`, то другой поток будет ждать, пока первый поток не освободит его. Таким образом, мы можем гарантировать, что в блоке `lock` будет выполняться только один поток.

Но почему мы не используем само поле `done` в качестве ресурса? Дело в том что сама контрукция является короткой записью для следующего кода

In [12]:
static bool done;
static readonly object locker = new object();

static void Main()
{
    var thread = new Thread(Go);
    thread.Start();
    Go();

    skip_in_console(thread);
}

static void Go()
{
    bool lockWasTaken = false;
    var temp = locker;
    try
    {
        Monitor.Enter(temp, ref lockWasTaken);
        {
            if (!done)
            {
                 Console.WriteLine("Done");
                 done = true;
            }
        }
    }
    finally
    {
        if (lockWasTaken)
        {
            Monitor.Exit(temp);
        }
    }
}

Main();

Done


Этот механизм называется **монитором** (monitor) и требует в качестве ресурса объект ссылочного типа. При этом, в случае, если бы мы передали в качестве ресурса поле `done`, монитром был бы захвачен объект, который бы создался с помощью механизма автоматического упаковывания (boxing) и который был бы разным для каждого из потоков.

Монитор - это внутренний механизм dotnet, и работает благодаря внутреннему способу хранения объектов, в котором каждый объект ссылочного типа имеет внутри себя индекс специальной таблицы, в которой хранятся все объекты, которые используются в качестве ресурса монитора.

![Монитор](./images/object.gif)

Упрощенно, процесс доступа к монитору можно представить следующим образом

![Монитор](./images/monitor.webp)

Еще одной особенностью этого механизма является то, что если один поток захватил монитор, то он может повторно захватить его, не вызывая блокировки. Это позволяет избежать блокировки в случае, если внутри блока `lock` вызывается метод, который сам использует блокировку. Например, следующий код не вызовет блокировку.

Кроме того, только тот поток, который захватил монитор, может его освободить. Поэтому, невозможно вызвать метод `Monitor.Exit` в другом потоке, если он не захватил ресурс.

Чтобы стало еще интереснее, давайте рассмотрим следующий пример

In [8]:
static bool done;
static object locker1 = new object();
static object locker2 = new object();

static void Main()
{
    var thread = new Thread(() =>
    {
        lock (locker1)
        {
            Thread.Sleep(1000);
            lock (locker2) // Deadlock
            {
                if (!done) { Console.WriteLine("Done"); done = true; }
            }
        }
    });
    thread.Start();

    lock (locker2)
    {
        Thread.Sleep(1000); // Deadlock
        lock (locker1)
        {
            if (!done) { Console.WriteLine("Done"); done = true; }
        }
    }
}

// Cancell command when locked
Main();

Error: Command cancelled.

Давайте рассмотрим пример поподробнее. Первый поток захватывает монитор `locker1`, затем он засыпает на 1 секунду. Второй поток захватывает монитор `locker2`, затем он засыпает на 1 секунду. После этого первый поток пытается захватить монитор `locker2`, а второй поток пытается захватить монитор `locker1`. Таким образом, оба потока ожидают освобождения ресурс, но никто не может их освободить, так как каждый поток захватил ресурс, который нужен другому потоку.

Такая ситуация называется **взаимной блокировкой** (deadlock). Программа при этом не завершается (зависает), так как оба потока находятся в состоянии ожидания (блокировки).

Тут следует сделать замечание о состояниях потоков. Поток может находиться в одном из следующих основных состояний (все возможные состояния можно найти в перечислении `ThreadState` и по ссылке https://docs.microsoft.com/en-us/dotnet/api/system.threading.threadstate?view=net-7.0).
 
 - `Aborted` Поток не выполняет работу, но его состояние еще не изменилось на Stopped.
 - `Running` Поток был запущен и выполняется в данный момент.
 - `Stopped` Поток был остановлен.
 - `Suspended` Поток был приостановлен.
 - `Unstarted` Метод *Start()* не был вызван для потока.
 - `WaitSleepJoin` Поток заблокирован. Это может произойти в результате вызова метода *Sleep(Int32)* или метода *Join()*, в результате запроса блокировки, например при вызове метода *Enter(Object)* или *Wait(Object, Int32, Boolean)* или в результате ожидания объекта синхронизации потока, такого как *ManualResetEvent*.

Текущее состояние потока можно получить с помощью свойства `Thread.ThreadState`. В нашем примере, оба потока находятся в состоянии `WaitSleepJoin` и не выполняют никакой работы. Состояние потока кроме методов, которые мы использовали, могут изменить методы `Abort()`, `Suspend()`, `Join()`, `Interrupt()`. Давайте рассмотрим их подробнее.

Для начала, давайте просто выполним метод `Go` на одном потоке.

In [13]:

static void Main()
{
    Thread thread = new Thread(Go);
    Console.WriteLine("Thread has started!");
    thread.Start();
    Console.WriteLine("\nThread has ended!");

    skip_in_console(thread);
}

static void Go()
{
    for (int i = 0; i < 10; i++)
    {
        Console.Write("y");
    }
}

Main();

Thread has started!

Thread has ended!
yyyyyyyyyy

Тут видно, что сообщение `Thread has ended!` появилось во время выполнения метода `Go`, но как дождаться завершения выполнения этого метода? Тут поможет метод `Join()` у потока `thread`. Вызов этого метода блокирует текущий поток до тех пор, пока поток `thread` не завершится.

In [ ]:
static void Main()
{
    Thread thread = new Thread(Go);
    thread.Start();

    thread.Join(); // Waits for the thread to finish

    Console.WriteLine("\nThread t has ended!");
}

static void Go()
{
    for (int i = 0; i < 10; i++)
    {
        Console.Write("y");
    }
}

Main();

Но каким образом выполнялся метод `Go()`, ведь текущий метод завершалcя во время его выполнения? Дело в том что поток, созданный нами с помощью конструктора `Thread(Go)`, является **основным** (foreground). Это означает, что процесс dotnet не завершится, пока не завешат работу все основные потоки. Если мы хотим, чтобы поток остановил свою работу после завершения основного потока, то нам нужно сделать его **фоновым** (background). Это можно сделать с помощью свойства `Thread.IsBackground`.

In [3]:
static void Main()
{
    Thread thread = new Thread(Go);
    thread.IsBackground = true; // Set the thread to work in background
    thread.Start();

    Console.WriteLine("\nThread t has ended!");
}

static void Go()
{
    for (int i = 0; i < 10; i++)
    {
        Console.Write("y");
    }
}

Main();


Thread t has ended!
yyyyyyyyyy

Исходя из стандартного вывода можно видеть, что программа завершилась, не дожидаясь завершения потока `thread`, хотя какая та чать работы была выполнена.

> Стоит заметить, что движок *.NET Interactive* не всегдя завершает работу, если главный поток не дождался завершения других основных потков. Поэтому мы вынужденно используем метод `skip_in_console` для того, чтобы завершить работу ячейки. В обычном приложении такого не происходит.

Еще один способ завершить работу приложения - принудительно завершить поток. Это можно сделать с помощью метода `Abort()`.

In [4]:
static void Main()
{
    Thread thread = new Thread(Go);
    thread.Start();

    thread.Abort(); // Waits for the thread to finish

    Console.WriteLine("\nThread t has ended!");
}

static void Go()
{
    for (int i = 0; i < 100; i++)
    {
        Console.Write("y");
    }
}

Main();

yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy

Error: System.PlatformNotSupportedException: Thread abort is not supported on this platform.
   at System.Threading.Thread.Abort()
   at Submission#5.Main()
   at Submission#5.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Однако, этот метод не рекомендуется использовать, так как он может привести к непредсказуемым последствиям. Например, если поток заблокирован, то он не сможет завершиться, пока не будет разблокирован. Также, если поток выполняет какую-то работу, то он может завершиться в любой момент, что может привести к непредсказуемым последствиям.

Для разблоктровки потока можно использовать метод `Interrupt()`, однако, этот метод не прерывает поток, если он выполняет какую-то работу. Начиная в версии .net 5 при вызове метода `Abort()` будет выброшено исключение `PlatformNotSupportedException` на текущем потоке. Поэтому, если вы хотите прервать поток, то вам нужно использовать специпльный механизм для этого. Одним из таких механизмов является использование спкциальных легковесных объектов, которые называются **токенами отмены** (cancellation tokens). Они позволяют прервать работу потока, если он выполняет какую-то работу.

In [2]:
static void Main()
{
    Thread thread = new Thread(new ParameterizedThreadStart(Go));

    // Create a cancellation token and cancel the thread
    CancellationToken cancellationToken = new CancellationToken(true);

    thread.Start(cancellationToken);

    thread.Join(); // Waits for the thread to finish

    Console.WriteLine("\nThread t has ended!");
}

static void Go(object state)
{
    for (int i = 0; i < 100; i++)
    {
        Console.Write("y");
    }
}

Main();

yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
Thread t has ended!


Сущность `CancellationToken` является структурой, которая переносит информацию о том, нужно ли отменить выполнение операции. Текущее сосотояние можно получить с помощью свойства `IsCancellationRequested`. Особенностью работы с этой структурой является то, что отмена операции лежит целиком на разработчике, т.е. сам по себе токен ничего отменить не может и не работает с потоками. Типичным способом отмены операции служит вызова метода `ThrowIfCancellationRequested`, который выбрасывает исключение `OperationCanceledException`, если поток нужно прервать.

In [ ]:
static void Main()
{
    Thread thread = new Thread(new ParameterizedThreadStart(Go));

    // Create a cancellation token and cancel the thread
    CancellationToken cancellationToken = new CancellationToken(true);

    thread.Start(cancellationToken);

    thread.Join(); // Waits for the thread to finish

    Console.WriteLine("\nThread t has ended!");
}

static void Go(object state)
{
    CancellationToken token = (CancellationToken)state;
    for (int i = 0; i < 100; i++)
    {
        token.ThrowIfCancellationRequested();
        Console.Write("y");
    }
}

Main();

Как мы видим, это приводит к тому, что поток завершается, но при этом выбрасывается исключение. Однако, для корректной работы программы, исключение лучше обработать.

In [ ]:
static void Main()
{
    Thread thread = new Thread(new ParameterizedThreadStart(Go));
    
    // Create a cancellation token and cancel the thread
    CancellationToken cancellationToken = new CancellationToken(true);

    try 
    {
        thread.Start(cancellationToken);
        thread.Join(); // Waits for the thread to finish
        Console.WriteLine("\nThread t has ended!");
    }
    catch (OperationCanceledException)
    {
        Console.WriteLine("\nOperation was cancelled!");
    }
    catch (Exception ex)
    {
        Console.WriteLine("\nUnhandled Exception: " + ex);
    }
}

static void Go(object state)
{
    CancellationToken token = (CancellationToken)state;
    for (int i = 0; i < 100; i++)
    {
        token.ThrowIfCancellationRequested();
        Console.Write("y");
    }
}

Main();

Как мы видим, обработка исключения на глоавном потоке ни к чему не приводит, дело в том, что исключения в dotnet существуют только в рамках одного потока. Поэтому, чтобы обработать исключение, нужно обрабатывать его в самом потоке.

In [4]:
static void Main()
{
    Thread thread = new Thread(Go);
    
    // Create a cancellation token and cancel the thread
    CancellationToken cancellationToken = new CancellationToken(true);

    try 
    {
        thread.Start(cancellationToken);
        thread.Join(); // Waits for the thread to finish
        Console.WriteLine("\nThread has ended!");
    }
    catch (Exception ex)
    {
        Console.WriteLine("\nUnhandled Exception: " + ex);
    }
}

static void Go(object state)
{
    CancellationToken token = (CancellationToken)state;
    try
    {
        for (int i = 0; i < 100; i++)
        {
            token.ThrowIfCancellationRequested();
            Console.Write("y");
        }
    }
    catch (OperationCanceledException)
    {
        Console.WriteLine("\nOperation was cancelled!");
    }
}

Main();


Operation was cancelled!

Thread has ended!


Как мы видим, поток завершился, не успев выполнить ни одной операции. Все потому, что мы создали токен сразу в отмененном состоянии. Обычно такого не происходит. Однако, токен сам по себе представляет такую структуру, которая не позволяет произвольно менять состояние. Для этой задачи обычно используют вспомогптельный класс `CancellationTokenSource`, который позволяет создавать токены и менять их состояние.

In [9]:
static void Main()
{
    Thread thread = new Thread(Go);
    
    // Create a cancellation token and cancel the thread
    var cancellationTokenSource = new CancellationTokenSource();
    cancellationTokenSource.CancelAfter(10);
    var cancellationToken = cancellationTokenSource.Token;

    try 
    {
        thread.Start(cancellationToken);
        thread.Join(); // Waits for the thread to finish
        Console.WriteLine("\nThread t has ended!");
    }
    catch (Exception ex)
    {
        Console.WriteLine("\nUnhandled Exception: " + ex);
    }
}

static void Go(object state)
{
    CancellationToken token = (CancellationToken)state;
    try
    {
        for (int i = 0; i < 1000; i++)
        {
            token.ThrowIfCancellationRequested();
            Console.Write("y");
        }
    }
    catch (OperationCanceledException)
    {
        Console.WriteLine("\nOperation was cancelled!");
    }
}

Main();

yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
Operation was cancelled!

Thread t has ended!


При помощи этого механизма можно создавать токены, которые можно отменять из других потоков. Для этого нужно вызвать метод `Cancel` у объекта `CancellationTokenSource`, либо использовать метод `CancelAfter`, который позволяет отменить токен через определенное время. Таким образом, используя один токен, можно управлять поведением нескольких потоков.

Далее мы рассмотрим другие механизмы взаимодействия между потоками.